In [4]:
import dask.dataframe as dd
import numpy as np
import pandas as pd

In [5]:
df_All_Path = r"df_all_fit_filterd.csv"

usecols = ['FIT_Test_Value', 'ALTC', 'EnvCount', 'Gene','Temperature', 'Radiation', 'FIT_Reference_Value', 'Date']


df = dd.read_csv(df_All_Path,assume_missing=True, usecols=usecols,dtype={'FIT_Test_Value': float,'ALTC': str,'EnvCount': str,'Gene': str, "Temperature":float , "Radiation":float ,
       "FIT_Correct_Value": float}).compute()

In [6]:
Corr_df = df.groupby(['ALTC', 'Gene']).apply(lambda x: x['FIT_Test_Value'].corr(x['FIT_Reference_Value'])).reset_index(name='Corr')

def calculate_rmse(group):
    actual = group['FIT_Reference_Value']
    predicted = group['FIT_Test_Value']
    mse = np.mean((actual - predicted) ** 2)
    rmse = np.sqrt(mse)
    return rmse

RMSE_df = df.groupby(['ALTC', 'Gene']).apply(calculate_rmse).reset_index(name='RMSE')

df_all = pd.merge(Corr_df,RMSE_df,how="inner",on = ["ALTC","Gene"])

In [7]:
df_all.to_csv(r"df_CorrRMSE.csv", index = False)